In [1]:
import numpy as np
import time
import sys
import os
import copy
from scipy.optimize import linprog 

sys.path.append(os.path.abspath("../src"))

from pyotc.otc_backend.policy_iteration.exact import exact_otc_pot
from pyotc.otc_backend.graph.utils import adj_to_trans, get_degree_cost
from pyotc.examples.stochastic_block_model import stochastic_block_model

/Users/bongsooyi/opt/anaconda3/envs/pyotc2/lib/python3.10/site-packages/scikits/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)


In [2]:
# Seed number
np.random.seed(1004)

In [3]:
# m = 5
# A1 = stochastic_block_model(
#     (m, m, m),
#     np.array(
#         [
#             [0.9, 0.1, 0.1],
#             [0.1, 0.9, 0.1],
#             [0.1, 0.1, 0.9],
#         ]
#     ),
# )
# A2 = stochastic_block_model(
#     (m, m, m),
#     np.array(
#         [
#             [0.9, 0.1, 0.1],
#             [0.1, 0.9, 0.1],
#             [0.1, 0.1, 0.9],
#         ]
#     ),
# )
# P1 = adj_to_trans(A1)
# P2 = adj_to_trans(A2)
# c = get_degree_cost(A1, A2)

# start = time.time()
# exp_cost, otc, stat_dist = exact_otc_pot(P1, P2, c)
# end = time.time()
# print(exp_cost, end - start)

In [3]:
m = 15
A1 = stochastic_block_model(
    (m, m, m, m),
    np.array(
        [
            [0.9, 0.1, 0.1, 0.1],
            [0.1, 0.9, 0.1, 0.1],
            [0.1, 0.1, 0.9, 0.1],
            [0.1, 0.1, 0.1, 0.9],
        ]
    ),
)
A2 = stochastic_block_model(
    (m, m, m, m),
    np.array(
        [
            [0.9, 0.1, 0.1, 0.1],
            [0.1, 0.9, 0.1, 0.1],
            [0.1, 0.1, 0.9, 0.1],
            [0.1, 0.1, 0.1, 0.9],
        ]
    ),
)
P1 = adj_to_trans(A1)
P2 = adj_to_trans(A2)
c = get_degree_cost(A1, A2)

start = time.time()
exp_cost, otc, stat_dist = exact_otc_pot(P1, P2, c)
end = time.time()
print(exp_cost, end - start)

1.093743196642743 138.9353609085083


In [19]:

import numpy as np
import scipy.sparse as sp
import ot

def computeot_pot(C, r, c):
    # Ensure r and c are numpy arrays
    r = np.array(r).flatten()
    c = np.array(c).flatten()

    # Compute the optimal transport plan and the cost using the ot.emd function
    lp_sol = ot.emd(r, c, C)
    lp_val = np.sum(lp_sol * C)

    return lp_sol, lp_val


def exact_tce_sparse(R_sparse, c):
    n = R_sparse.shape[0]
    c = np.reshape(c, (n, -1))
    I = sp.eye(n, format='csr')

    zero = sp.csr_matrix((n, n))
    A = sp.bmat([
        [I - R_sparse, zero, zero],
        [I, I - R_sparse, zero],
        [zero, I, I - R_sparse]
    ], format='csr')
    rhs = np.concatenate([np.zeros((n, 1)), c, np.zeros((n, 1))])

    permc_specs = ['COLAMD', 'MMD_ATA', 'MMD_AT_PLUS_A', 'NATURAL']
    solution = None 
    for spec in permc_specs:
        try:
            #current_solution = sp.linalg.spsolve(A, rhs, permc_spec=spec)
            current_solution = sp.linalg.lsmr(A, rhs, atol=1e-8, btol=1e-8)[0]
            if not np.any(np.abs(current_solution) > 1e15):
                solution = current_solution
                break 
            else:
                print(f"Solution with {spec} contains large values, trying next spec.")
        except ValueError as e:
            print(f"spsolve with {spec} encountered an error: trying next spec.")

    if solution is None:
        raise RuntimeError("Failed to find a stable solution with any of the provided permc_specs for sp.linalg.spsolve solver.")
    
    g = solution[:n]
    h = solution[n:2*n]
    return g, h

def setup_ot_sparse_fixed(f, Px, Py, Pz):
    dx = Px.shape[0]
    dy = Py.shape[0]
    f_mat = np.reshape(f, (dx, dy))

    for x_row in range(dx):
        for y_row in range(dy):
            dist_x = Px[x_row, :]
            dist_y = Py[y_row, :]
            # degenerate distribution check
            if np.any(dist_x == 1) or np.any(dist_y == 1):
                sol = np.outer(dist_x, dist_y)
            else:
                sol, _ = computeot_pot(f_mat, dist_x, dist_y) 
            idx = dy * x_row + y_row
            sol_flat = sol.flatten()
            for j in np.nonzero(sol_flat)[0]:
                Pz[idx, j] = sol_flat[j]
    return Pz

def exact_tci_sparse(g, h, P0, Px, Py): 
    dx, dy = Px.shape[0], Py.shape[0]
    Pz = sp.lil_matrix((dx * dy, dx * dy))
    g_const = np.max(g) - np.min(g) <= 1e-3

    if not g_const:
        Pz = setup_ot_sparse_fixed(g, Px, Py, Pz) 
        if np.max(np.abs(P0.dot(g) - Pz.dot(g))) <= 1e-7:
            Pz = P0.copy()
        else:
            return Pz

    Pz = setup_ot_sparse_fixed(h, Px, Py, Pz) 
    if np.max(np.abs(P0.dot(h) - Pz.dot(h))) <= 1e-4:
        Pz = P0.copy()

    return Pz


def get_best_stat_dist_sparse(P_sparse, c):
    n = P_sparse.shape[0]
    c = np.reshape(c, (n, -1))

    # Construct Aeq in sparse format
    eye_n = sp.eye(n, format='csr')
    row_sum = sp.csr_matrix(np.ones((1, n)))  # sum(x) = 1 constraint
    Aeq_sparse = sp.vstack([P_sparse.transpose() - eye_n, row_sum], format='csr')
    beq = np.concatenate((np.zeros((n, 1)), [[1]]), axis=0)
    bounds = [(0, None)] * n

    # Solve linear program (method='highs' supports sparse)
    res = linprog(c,
                A_eq=Aeq_sparse,
                b_eq=beq,
                bounds=bounds,
                method='highs')
    if not res.success:
        raise RuntimeError("Linear program failed: " + res.message)

    return res.x, res.fun

def get_stat_dist_sparse(P_sparse, max_iter=10000, tol=1e-10):
    """
    Computes the stationary distribution of a sparse transition matrix using power iteration.

    Args:
        P_sparse (sp.spmatrix): (n x n) row-stochastic transition matrix
        max_iter (int): max number of iterations
        tol (float): convergence tolerance

    Returns:
        pi (np.ndarray): stationary distribution of shape (n,)
    """
    n = P_sparse.shape[0]
    pi = np.ones(n) / n  # initial uniform distribution

    for _ in range(max_iter):
        pi_new = pi @ P_sparse
        if np.linalg.norm(pi_new - pi, ord=1) < tol:
            break
        pi = pi_new

    pi /= np.sum(pi)  # ensure normalization
    return pi



def exact_otc_pot_sparse(Px, Py, c, get_best_sd=True, max_iter=100):
    start = time.time()
    dx, dy = Px.shape[0], Py.shape[0]
    P = sp.kron(sp.csr_matrix(Px), sp.csr_matrix(Py), format='csr')

    for iter in range(max_iter):
        print("Iteration:", iter)

        P_old = P.copy()
        g, h = exact_tce_sparse(P, c)
        P = exact_tci_sparse(g, h, P_old, Px, Py) #, forbidden_set)

        #print(np.max(np.abs(P.toarray()-P_old.toarray())))
        #if np.max(np.abs(P.toarray()-P_old.toarray())) <= 1e-10:
        if (P != P_old).nnz == 0:
            if get_best_sd:
                stat_dist, exp_cost = get_best_stat_dist_sparse(P, c)
                stat_dist = np.reshape(stat_dist, (dx, dy))
            else:
                stat_dist = get_stat_dist_sparse(P)
                stat_dist = np.reshape(stat_dist, (dx, dy))
                exp_cost = g[0].item()
            end = time.time()
            print(f"Convergence reached in {iter} iterations, took {end - start:.4f} seconds")
            return float(exp_cost), P, stat_dist

    return None, None, None

In [5]:
exp_cost_sparse, otc_sparse, stat_dist_sparse = exact_otc_pot_sparse(P1, P2, c)
print(exp_cost_sparse)


Iteration: 0
Solution with COLAMD contains large values, trying next spec.
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Solution with COLAMD contains large values, trying next spec.
Convergence reached in 4 iterations, took 26.9875 seconds
0.7103069746935031


In [23]:
exp_cost_sparse, otc_sparse, stat_dist_sparse = exact_otc_pot_sparse(P1, P2, c)
print(exp_cost_sparse)


Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Convergence reached in 4 iterations, took 10.1033 seconds
0.7103069747735357


In [15]:
a, _, not_best = exact_otc_pot_sparse(P1, P2, c, get_best_sd=False)
print(a)

Iteration: 0
Solution with COLAMD contains large values, trying next spec.
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Solution with COLAMD contains large values, trying next spec.
Convergence reached in 4 iterations, took 20.5820 seconds
0.7103069746934912


In [20]:
Px, Py = P1, P2
c = get_degree_cost(A1, A2)
dx, dy = Px.shape[0], Py.shape[0]
n = dx * dy

P = sp.kron(sp.csr_matrix(Px), sp.csr_matrix(Py), format='csr')

P_old = P.copy()

g, h = exact_tce_sparse(P, c)
P = exact_tci_sparse(g, h, P_old, Px, Py) #, forbidden_set)

print(np.max(np.abs(P.toarray()-P_old.toarray())))

0.109375


In [21]:
g

array([5.34901515, 5.34900873, 5.34901692, ..., 5.34898946, 5.34899069,
       5.34898948])

In [22]:
h

array([ 9.54502668,  0.16226404, -2.80037941, ..., -5.30223915,
       -4.62552954, -1.71026877])

In [10]:
h

array([ 9.537439  ,  0.15749456, -2.80812496, ..., -5.29758017,
       -4.62067663, -1.70579202])

In [18]:
P.toarray()[10][200:400]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.08333333, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [7]:
P_old = P.copy()
g, h = exact_tce_sparse(P, c)
P = exact_tci_sparse(g, h, P_old, Px, Py) 
print(np.max(np.abs(P.toarray()-P_old.toarray())))

0.125


In [8]:
P_old = P.copy()
g, h = exact_tce_sparse(P, c)
P = exact_tci_sparse(g, h, P_old, Px, Py) 
print(np.max(np.abs(P.toarray()-P_old.toarray())))

0.125


In [28]:
P_old = P.copy()
g, h = exact_tce_sparse(P, c)
P = exact_tci_sparse(g, h, P_old, Px, Py) 
print(np.max(np.abs(P.toarray()-P_old.toarray())))

0.125


In [29]:
g

array([0., 0., 0., ..., 0., 0., 0.])

In [31]:
h[100]

37.784782653369895

In [11]:
R_sparse = P
n = R_sparse.shape[0]
c = np.reshape(c, (n, -1))
I = sp.eye(n, format='csr')

zero = sp.csr_matrix((n, n))
A = sp.bmat([
    [I - R_sparse, zero, zero],
    [I, I - R_sparse, zero],
    [zero, I, I - R_sparse]
], format='csr')

rhs = np.concatenate([np.zeros((n, 1)), c, np.zeros((n, 1))])
solution = sp.linalg.spsolve(A, rhs, permc_spec='MMD_ATA')

print(solution[:n])
print(solution[n:2*n])
print(solution[2*n:3*n])

[0. 0. 0. ... 0. 0. 0.]
[48.33036312 34.21570604 33.98193715 ... 42.26738245 43.53125
 41.85336382]
[-1.75425504e+18 -1.75425504e+18 -1.75425504e+18 ... -1.75425504e+18
 -1.75425504e+18 -1.75425504e+18]


In [12]:
from scipy.sparse.linalg import spsolve, MatrixRankWarning
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("error", MatrixRankWarning)  # 경고를 에러로 바꿈
    try:
        x = sp.linalg.spsolve(A, rhs, permc_spec='MMD_ATA')
    except MatrixRankWarning as e:
        print("⚠️ MatrixRankWarning 발생:", e)


In [19]:
-x[-1]  > 1e+15

True

In [9]:
solution = sp.linalg.spsolve(A, rhs, permc_spec='MMD_ATA', use_umfpack=False)

print(solution[:n])
print(solution[n:2*n])
print(solution[2*n:3*n])

NameError: name 'A' is not defined

In [7]:
Px, Py = P1, P2
c = get_degree_cost(A1, A2)
dx, dy = Px.shape[0], Py.shape[0]
n = dx * dy
forbidden_set = get_forbidden_indices_from_kronecker(Px, Py)

P = sp.kron(sp.csr_matrix(Px), sp.csr_matrix(Py), format='csr')

P_old = sp.csr_matrix(np.ones((n, n)))
max_iter = 100
tol = 1e-10

diff = np.max(np.abs(P.toarray() - P_old.toarray()))
if diff <= tol:
    print('break')
P_old = copy.deepcopy(P)

g, h = exact_tce_sparse(P, c)


# P = exact_tci_sparse(g, h, P_old, Px, Py, forbidden_set)


#if (P != P_old).nnz == 0:


In [27]:
R_sparse = P
n = R_sparse.shape[0]
c = np.reshape(c, (n, -1))
I = sp.eye(n, format='csr')

zero = sp.csr_matrix((n, n))

A = sp.bmat([
    [I - R_sparse, zero, zero],
    [I, I - R_sparse, zero],
    [zero, I, I - R_sparse]
], format='csr')

rhs = np.concatenate([np.zeros((n, 1)), c, np.zeros((n, 1))])
solution = sp.linalg.spsolve(A, rhs)

In [30]:
np.linalg.inv(A.toarray()) @ rhs

array([[  4.88140268],
       [  4.88140268],
       [  4.88140268],
       ...,
       [230.25736019],
       [241.08122875],
       [245.17754113]], shape=(1200, 1))

In [32]:
solution = sp.linalg.spsolve(A, rhs)
solution

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -6.61183378e+17, -6.61183378e+17, -6.61183378e+17], shape=(1200,))